In [1]:
import pandas as pd
import numpy as np
import os
import glob
from PIL import Image, ImageOps
import cv2

# Load data

In [2]:
path_train = glob.glob("../../data/globular_dataset_ds9/train/images/*")
# path_train_labels = glob.glob("../../data/globular_dataset_ds9/train/labels/*")
# path_test = glob.glob("../../data/globular_dataset_ds9/test/images/*")
# path_test_labels = glob.glob("../../data/globular_dataset_ds9/test/labels/*")

In [105]:
def load_data(path, gray_mode = False, normalize = False):
  images_list = []
  images_name = []
  labels_list = []
  for i in path:
    if gray_mode:
      image = cv2.imread(i, 0)
    else:
      image = cv2.imread(i, 1)

    if normalize:
      # normalize the image to 0-1
      image = image / 255.0
    else:
      pass

    # save as a numpy array in the list
    images_list.append(image)
    # save the name of the image
    images_name.append(os.path.basename(i))
    # save the label of the image
    labels_list.append(os.path.basename(i).split("_")[0])
  return np.array(images_list), np.array(images_name), np.array(labels_list)

In [106]:
# images_train, images_train_name, labels_train = load_data(path_train, gray_mode = True, normalize = True)

In [3]:
data_train = np.array([np.array(ImageOps.grayscale(Image.open(fname))) for fname in path_train])

In [5]:
# normalize
data_train = data_train / 255.0

# Build model

## Convolutional autoencoder

In [44]:
# build cnn autoencoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
import tensorflow as tf


w = data_train[0].shape[0]
h = data_train[0].shape[1]
channel = 1
input_img = Input(shape=(w, h, channel))  # adapt this if using `channels_first` image data format

x = Conv2D(4, (3, 3), activation='relu', padding='same')(input_img)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(16, (3,3), activation='relu', padding='same')(x)
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
encoder = layers.MaxPooling2D((2, 2), padding="same", name='encoder')(x)

x = layers.Conv2DTranspose(16, (3, 3), strides=2, activation="relu", padding="same")(encoder) # model should learn what is the best upsampling for the job from transposed convolutions
x = layers.Conv2DTranspose(8, (3, 3), strides=2, activation="relu", padding="same")(x) # upsampling using transposed convolutions
x = layers.Conv2DTranspose(4, (3, 3), strides=1, activation="relu", padding="same")(x)
decoder = layers.Conv2D(channel, (3, 3), activation="sigmoid", padding="same", name='decoder')(x)


autoencoder = Model(input_img, decoder)
opt = tf.optimizers.Adam(learning_rate=0.01)
autoencoder.compile(optimizer=opt, loss='MeanSquaredError')
autoencoder.summary()

Model: "functional_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1024, 1024, 1)]   0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 1024, 1024, 4)     40        
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 1024, 1024, 8)     296       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 512, 512, 8)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 512, 512, 16)      1168      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 512, 512, 32)      4640      
_________________________________________________________________
encoder (MaxPooling2D)       (None, 256, 256, 32)    

In [45]:
autoencoder.fit(
    x=data_train,
    y=data_train,
    epochs=1,
    batch_size=4,
)

193/193 [==============================] - 49s 253ms/step - loss: 0.0556


In [46]:
z = data_train[0].reshape(1, w, h, channel)

In [47]:
z.shape

(1, 1024, 1024, 1)

In [48]:
# predict the images by encoder
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('encoder').output)
encoder_output = encoder.predict(z)



In [49]:
encoder_output.shape

(1, 256, 256, 32)

In [73]:
# predict encoder_output by decoder
decoder = Model(inputs=autoencoder.get_layer('encoder').inputs, outputs=autoencoder.get_layer('decoder').output)
decoder.summary()

AttributeError: 'MaxPooling2D' object has no attribute 'inputs'

In [ ]:
decoder_output.shape

In [ ]:
plt.imshow(decoder_output.reshape(1024, 1024, 1), cmap='gray') 
plt.show()  # display it